In [1]:
import numpy as np
import pandas as pd
import faiss
import pickle
from sklearn.preprocessing import MinMaxScaler
import tqdm

In [2]:
minmaxscaler = MinMaxScaler((-50, 50))

In [3]:
drop_columns = ["6", "44", "70", "21", "25", "33", "59", "65"]

In [4]:
# data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)
# data_validation = pd.read_csv("../Masterskaya-2/data/validation.csv", index_col=0)
# data_validation_answer = pd.read_csv("../Masterskaya-2/data/validation_answer.csv", index_col=0)["Expected"]

In [5]:
data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)
data_validation = pd.read_csv("../Masterskaya-2/data/validation.csv", index_col=0)
data_validation_answer = pd.read_csv("../Masterskaya-2/data/validation_answer.csv", index_col=0)["Expected"]

data_base = data_base.drop(drop_columns, axis=1)
data_validation = data_validation.drop(drop_columns, axis=1)

In [6]:
base_index = {k: v for k, v in enumerate(pd.DataFrame(data_base).index.to_list())}

In [7]:
data_base = minmaxscaler.fit_transform(data_base)
data_validation = minmaxscaler.transform(data_validation)

In [8]:
minmaxscaler = None

In [9]:
with open('../Masterskaya-2/models/model.pkl', 'rb') as fid:
    model = pickle.load(fid)
    fid.close()

In [10]:
data_validation = pd.DataFrame(model.predict(data_validation))

In [11]:
model = faiss.read_index("../Masterskaya-2/models/model.faiss")
model = faiss.index_cpu_to_all_gpus(model)
# model.nprobe = 5

In [12]:
vecs, idx = model.search(np.ascontiguousarray(data_validation).astype('float32'), 2000)

In [13]:
model = None

In [14]:
acc = 0
for target, el in zip(data_validation_answer.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
print(100 * acc / len(idx))

86.788


In [15]:
# vectors_index = []
# for target, el in tqdm.tqdm(zip(data_validation_answer.values.tolist(), idx.tolist())):             
#     data_list = [base_index[r] for r in el]
#     try:
#         index = data_list.index(target)
#         data_list.pop(index)
#     except Exception:
#         continue
#     vectors_index.append(data_list)


In [16]:
# base_index = None
# vecs = None
# idx = None

In [17]:
# data_base = pd.read_csv("../Masterskaya-2/data/base.csv", index_col=0)

In [18]:
# vectors = []
# for i in vectors_index:
#     matrix = []
#     for el in i:
#         matrix.append(data_base.loc[el].to_list())
#     vectors.append(matrix)

In [19]:
# data_base = None
# vectors_index = None

In [20]:
# df = pd.DataFrame(columns=[x for x in range(0, 72)])
# for i in tqdm.tqdm(range(len(vectors))):
#     for el in range(len(vectors[i])):
#         df.loc[len(df.index)] = data_validation.loc[i]

In [21]:
# df.to_csv("./data/data_prediction_ranker_serched_vectors.csv")